In [2]:
import torch
import torchvision.models as models
from torch import nn

class CNNEncoder(nn.Module):
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.cnn = models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()  # Loại bỏ lớp fully connected cuối cùng

    def forward(self, images):
        features = self.cnn(images)
        return features


In [3]:
from transformers import BartForConditionalGeneration, BartTokenizer

class EventExtractionModel(nn.Module):
    def __init__(self, encoder, tokenizer):
        super(EventExtractionModel, self).__init__()
        self.encoder = encoder
        self.tokenizer = tokenizer
        self.decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

    def forward(self, images, input_text=None):
        features = self.encoder(images)
        outputs = self.decoder(input_ids=input_text, encoder_outputs=features)
        return outputs


In [4]:
from torch.optim import Adam

model = EventExtractionModel(encoder=CNNEncoder(), tokenizer=BartTokenizer.from_pretrained('facebook/bart-base'))
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 80.9MB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        images, captions = batch
        optimizer.zero_grad()
        outputs = model(images, captions)
        loss = criterion(outputs.logits, captions)
        loss.backward()
        optimizer.step()


NameError: name 'num_epochs' is not defined

In [6]:
model.eval()
with torch.no_grad():
    for batch in test_loader:
        images, captions = batch
        outputs = model(images)
        # Đánh giá kết quả bằng cách so sánh với captions thực tế


NameError: name 'test_loader' is not defined